In [2]:
import pandas as pd
import numpy as np
import pandas_gbq
import datetime
from dateutil.relativedelta import relativedelta
from gspread_pandas import Spread, conf

In [39]:
from ast import literal_eval

In [3]:
# Credenciales
cred = conf.get_config('C:\\Users\\santiago.curat\\Pandas\\PEYA', 'PedidosYa-8b8c4d19f61c.json')

In [4]:
# Mapeo Campañas
sheet_id = '1d4cMi-V04c80Dpen4S70Q7zYqqho3ZBIVctlRIBLwiw'
wks_name = 'Mapeo Campañas'
sheet = Spread(sheet_id, wks_name, config=cred)
mapeo = sheet.sheet_to_df(index=0,header_rows=1)

In [5]:
# Dropeo las columnas vacias
mapeo.drop([''],axis=1,inplace=True)
# Divido el DF en varios
cols = len(mapeo.columns)
dfs = []
for i in range(int(cols/3)):
    dfs.append(mapeo.iloc[:,i*3:i*3+3].copy())
# Borro las filas vacias de cada DF
dfs = [x[x['Columna'] != ''] for x in dfs]
dfs = pd.concat(dfs,ignore_index=True).copy()

In [6]:
# Creo un diccionario de variables
records = {}
filters = dfs['Columna'].unique().tolist()
for i in filters:
    records[i] = {}
# Completo el diccionario de variables
for i in records:
    records[i] = dfs[dfs['Columna'] == i].set_index(['Search']).to_dict()['Value']

In [7]:
# Defino la funcion de filtros
def filtros(cam,filtro):
    try:
        dicc = records[filtro]
    except:
        return 'No Existe Filtro'
    flag = 0
    try:
        for i in dicc:
            if i in str(cam):
                val = dicc[i]
                flag = 1
                break
        if flag == 0:
            val = dicc[list(dicc.keys())[-1]]
        return val
    except:
        return 'Error en Comparacion'

In [88]:
# IVA
sheet_id = '1d4cMi-V04c80Dpen4S70Q7zYqqho3ZBIVctlRIBLwiw'
wks_name = 'IVA'
sheet = Spread(sheet_id, wks_name, config=cred)
iva = sheet.sheet_to_df(index=0,header_rows=1)

In [89]:
# Listas
sheet_id = '1d4cMi-V04c80Dpen4S70Q7zYqqho3ZBIVctlRIBLwiw'
wks_name = 'Listas'
sheet = Spread(sheet_id, wks_name, config=cred)
listas = sheet.sheet_to_df(index=0,header_rows=1)

In [90]:
# Reemplazo los vacíos por 1
iva.replace([''],0,inplace=True)
# Paso los valores a mayúsculas
iva['Country'] = iva['Country'].str.upper()
# Cambio los países con acentos
acentos = literal_eval(listas[listas['Tipo'] == 'Paises']['Lista'].values[0])
def cambiar_paises(cou):
    flag = 0
    try:
        for i in acentos:
            if cou == i:
                val = acentos[i]
                flag = 1
        if flag == 0:
            val = cou
        return val
    except:
        return cou
iva['Country'] = iva['Country'].apply(cambiar_paises)
# Doy formato a las columnas
iva[['IVA','Extra']] = iva[['IVA','Extra']].astype(float)

In [111]:
def func_iva(df):
    try:
        df['Country'] = df['Country'].str.upper()
        df = df.merge(iva,on=['Country'],how='left').copy()
        df['IVA'].replace([np.nan,np.inf,-np.inf],0,inplace=True)
        df['Extra'].replace([np.nan,np.inf,-np.inf],1,inplace=True)
        return df
    except:
        if 'Country' in list(df.columns):
            print('Surgio otro Error')
            return df.copy()
        else:
            print('No existe la Columna Country')
            return df.copy()

In [117]:
t = pd.DataFrame.from_dict({'Country':['ARGENTINA','CHILE','PERÚ'],
                            'Campaign':[1,2,3]})

In [119]:
filters

['BENEFICIOS',
 'TIPOS',
 'SEGMENTOS',
 'NIVELES',
 'CAMPAIGNS',
 'AUTOMATIONS',
 'BUDGET']

In [124]:
t['Beneficios'] = t['Campaign'].apply(lambda x: filtros(x,'BENEFICIOS'))